# TARA Ontology: SPARQL Query Examples
This notebook is created to demonstrate the utility of TARA ontologies through a set of SPARQL queries. To use this notebook you will need to install [jupyter notebook](https://jupyter.org/install) with [SPARQL kernel](https://github.com/paulovn/sparql-kernel).
* It is recommended that you have the basic understanding of the TARA Acupoints Ontology based on the documentation [linked here](https://github.com/smtifahim/TARA-Ontology-Repository/blob/master/ontology-files/generated/readme.md).

## Set Up the Stardog SPARQL Endpoint for TARA Ontology
To run the example queries, please set up the Stardog SPARQL Endpoint first, by running the following magics.

In [1]:
%endpoint https://sd-c1e74c63.stardog.cloud:5820/TARA-Acupoints/query
%auth basic TARA qy8CLEJXrwVqKNsC
%format any
%display table
%show 16

Endpoint set to: https://sd-c1e74c63.stardog.cloud:5820/TARA-Acupoints/query
HTTP authentication: method=basic, user=TARA, passwd set
Request format: ANY
Display: table
Result maximum size: 16

### Q1: List all acupoints along with their labels, synonyms, and Chiense names

In [2]:
# To save the query result in JSON format uncomment the following two lines. Specify the location and name of the JSON output after %outfile
# %format JSON
# %outfile sparql-results/Q1-Results.json

# List all acupoints with their label, synonyms, and chinese names.

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
# PREFIX TARA: <http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#>

SELECT DISTINCT ?acupoint_curie ?acupoint_label ?synonym ?chinese_name
WHERE 
{
    ## Partial match for an acupoint either directly or synonymously. Uncomment the following two lines.
    #VALUES ?search_term { "fengmen" }
    #  FILTER ( REGEX(?acupoint_label, ?search_term, "i") || REGEX(?synonym, ?search_term, "i") || REGEX(?chinese_name, ?search_term, "i") )

    ?acupoint_iri rdfs:subClassOf+/rdfs:label 'Acupoint'.
    ?acupoint_iri rdfs:label ?acupoint_label.
    
    OPTIONAL { ?acupoint_iri TARA:hasSynonym ?synonym.}
    OPTIONAL { ?acupoint_iri TARA:hasChineseName ?chinese_name.}

    # Exclude generic categorizations like Acupoint of the X Meridian, Meridian Acupoint, Special Points, etc.
    FILTER (!regex(str(?acupoint_label), 'Acupoint of|Meridian Acupoint|Point of|Point|Extra Acupoint'))
    
    # To have the curie form of the acupoint_iri in the query result. 
    # Jupyter's SPARQL kernel does not return the curie form by default.
    BIND (REPLACE(STR(?acupoint_iri), "http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#", 
                                      "TARA:") AS ?acupoint_curie)
}
ORDER BY ?acupoint_label
LIMIT 999

acupoint_curie,acupoint_label,synonym,chinese_name
TARA:TARA_3480360,Anmian,,
TARA:TARA_1460220,BL 1,UB 1,Jingming
TARA:TARA_1460220,BL 1,Bladder 1,Jingming
TARA:TARA_0242146,BL 10,UB 10,Tianzhu
TARA:TARA_0242146,BL 10,Bladder 10,Tianzhu
TARA:TARA_1985889,BL 11,UB 11,Dazhu
TARA:TARA_1985889,BL 11,Influential Point of Bone,Dazhu
TARA:TARA_1985889,BL 11,Bladder 11,Dazhu
TARA:TARA_5468609,BL 12,UB 12,Fengmen
TARA:TARA_5468609,BL 12,Bladder 12,Fengmen


### Q2: List all meridans with their labels and synonyms

In [3]:
# List all meridans with their labels and synonyms.

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX TARA: <http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#>

SELECT DISTINCT ?meridian_curie ?meridian_label ?synonym
WHERE 
{
    ## Partial match of the meridan either directly or synonymously;
    # VALUES ?search_term { "Du" }
    # FILTER (REGEX(?meridian_label, ?search_term, "i") || REGEX(?synonym, ?search_term, "i"))


    ?meridian_iri rdfs:subClassOf+/rdfs:label "Meridian".
    ?meridian_iri rdfs:label ?meridian_label.
    
    OPTIONAL { ?meridian_iri TARA:hasSynonym ?synonym. }
    OPTIONAL { ?meridian_iri TARA:hasDesignatedOrgan ?associated_organ. }

    # Exclude generic categorization classes
    FILTER (!regex(str(?meridian_label), 'Main Meridian|Extra Meridian'))

    BIND ( REPLACE(STR(?meridian_iri), 
           "http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#", 
           "TARA:") AS ?meridian_curie)
}
ORDER BY ?meridina_label
limit 999

meridian_curie,meridian_label,synonym
TARA:TARA_6789717,Conception Vessel,CV
TARA:TARA_6789717,Conception Vessel,RN
TARA:TARA_6789717,Conception Vessel,Ren Channel
TARA:TARA_7525476,Bladder Meridian,BL
TARA:TARA_7525476,Bladder Meridian,Bladder Meridian of Foot-Taiyang
TARA:TARA_7525476,Bladder Meridian,meridian of the bladder
TARA:TARA_7525476,Bladder Meridian,UB
TARA:TARA_7525476,Bladder Meridian,meridan of the urinary bladder
TARA:TARA_7525476,Bladder Meridian,urinary bladder meridian
TARA:TARA_6221269,Stomach Meridian,ST


### Q3: List all the acupoints with meridans, locational description, acupuncture method, and reference.

In [4]:
# List all the acupoints with meridans, locational description, acupuncture method, and reference.

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX TARA: <http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#>

SELECT DISTINCT 
?acupoint_curie ?acupoint_label ?meridian 
?location_description ?acupuncture_method ?reference
WHERE 
{
    ## Partial match for an acupoint search
    #  VALUES ?search_term { "LI" }
    #  FILTER ( REGEX(?acupoint_label, ?search_term, "i") || REGEX(?meridian, ?search_term, "i") )

    ?acupoint_iri rdfs:subClassOf+/rdfs:label "Acupoint".
    ?acupoint_iri rdfs:label ?acupoint_label.
    
    OPTIONAL { ?acupoint_iri TARA:hasMeridian/rdfs:label ?meridian. }
    OPTIONAL { ?acupoint_iri TARA:hasLocationalDescription ?location_description. }
    OPTIONAL { ?acupoint_iri TARA:hasMethodDescription ?acupuncture_method. }
    OPTIONAL { ?acupoint_iri TARA:hasReference ?reference. }

    # Exclude generic categorization classes like 'Acupoint of the X Meridian'. Meridian Acupoint, 'Special Points', etc.
    FILTER (!regex(str(?acupoint_label), 'Acupoint of|Meridian Acupoint|Point of|Point|Extra Acupoint'))

    BIND (REPLACE(STR(?acupoint_iri), 
          "http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#", 
          "TARA:") AS ?acupoint_curie)
}
ORDER BY ?acupoint_label
limit 999

acupoint_curie,acupoint_label,meridian,location_description,acupuncture_method,reference
TARA:TARA_3480360,Anmian,,Midpoint between Yifeng (TE 17) and Fengchi (GB 20).,Puncture perpendicularly 0.5-0.8 inch.,"Chapter 10 of ""Chinese Acupuncture and Moxibustion” (1999) ISBN-13: 978-7119017599"
TARA:TARA_1460220,BL 1,Bladder Meridian,"On the face, in the depression between the superomedial parts of the inner canthus of the eye and the medial wall of the orbit. Note: When the eye is closed, BL1 is located in the depression 0.1 B-cun superior and medial to the inner canthus of the eye.","Ask the patient to close his eyes when pushing gently the eyeball to the lateral side. Puncture slowly perpendicularly 0.3-0.7 inch along the orbital wall. It is not advisable to twist of lift and thrust the needle vigorously. To avoid bleeding, press the puncturing site for a few seconds after withdrawal of the needle. Moxibustion is forbidden.","WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 100"
TARA:TARA_0242146,BL 10,Bladder Meridian,"In the posterior region of the neck, at the same level as the superior border of the spinous process of the second cervical vertebra (C2), in the depression lateral to the trapezius muscle.",Puncture perpendicularly 0.5-0.8 inch.,"WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 104"
TARA:TARA_1985889,BL 11,Bladder Meridian,"In the upper back region, at the same level as the inferior border of the spinous process of the first thoracic vertebra (T1), 1.5 B-cun lateral to the posterior median line.",Puncture obliquely 0.5-0.7 inch. Moxibustion is applicable.,"WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 105"
TARA:TARA_5468609,BL 12,Bladder Meridian,"In the upper back region, at the same level as the inferior border of the spinous process of the second thoracic vertebra (T2), 1.5 B- cun lateral to the posterior median line.",Puncture obliquely 0.5-0.7 inch. Moxibustion is applicable.,"WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 105"
TARA:TARA_6050541,BL 13,Bladder Meridian,"In the upper back region, at the same level as the inferior border of the spinous pro- cess of the third thoracic vertebra (T3), 1.5 B-cun lateral to the posterior median line.",Puncture obliquely 05-0.7 inch. Moxibustion is applicable.,"WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 106"
TARA:TARA_9523348,BL 14,Bladder Meridian,"In the upper back region, at the same level as the inferior border of the spinous process of the fourth thoracic vertebra (T4), 1.5 B-cun lateral to the posterior median line.",Puncture obliquely 0.5-0.7 inch. Moxibustion is applicable.,"WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 106"
TARA:TARA_5797983,BL 15,Bladder Meridian,"In the upper back region, at the same level as the inferior border of the spinous pro- cess of the fifth thoracic vertebra (T5), 1.5 B-cun lateral to the posterior median line.",Puncture obliquely 0.5-0.7 inch. Moxibustion is applicable.,"WHO standard acupuncture point locations in the Western Pacific Region. World Health Organization. Regional Office for the Western Pacific, 2008. Page 107"
TARA:TARA_1104010,BL 16,Bladder Meridian,"In the upper back region, level with the inferior border of the spinous process of the sixth thoracic vertebra (T6), 1.5 B-cun lateral to the posterior median line.",Puncture obliquely 0.5-0.7 inch. Moxibustion is applicable.,"WHO standard acupuncture 

### Q4: List all the acupoints along with their meridians and special point roles 

In [8]:
# List all the acupoints along with their meridians and special point roles

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX TARA: <http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#>

SELECT DISTINCT
?acupoint_curie ?acupoint_label ?meridian
?special_point_role ?special_point_category

WHERE 
{
    # FILTER (?acupoint = 'LU 9').
    # FILTER (?meridian = 'Lung Meridian').
     FILTER (?special_point_category = 'Front-Mu Point').
    # FILTER (?special_point_category = 'Xi-Cleft Point').

    OPTIONAL {?acupoint_iri TARA:hasMeridian/rdfs:label ?meridian.}
    ?acupoint_iri rdfs:subClassOf*/rdfs:label "Meridian Acupoint".
    ?acupoint_iri rdfs:label ?acupoint_label.

    OPTIONAL
    {
        ?acupoint_iri rdfs:subClassOf+ ?special_point_iri.        
        ?special_point_iri rdfs:subClassOf+ ?special_point_category_iri.
        ?special_point_category_iri rdfs:subClassOf+/rdfs:label 'Special Point'.
         
        ?special_point_iri rdfs:label ?special_point_role. 
        ?special_point_category_iri rdfs:label ?special_point_category.
    }

   # Exclude generic categorization classes like 'Acupoint of the X Meridian'. Meridian Acupoint, 'Special Points', etc.
    FILTER (!regex(str(?acupoint_label), 'Acupoint of|Meridian Acupoint|Point of|Point|Extra Acupoint'))
    FILTER (!regex(str(?acupoint_label), 'Male|Female'))
    
    # To have the curie form of the acupoint_iri in the query result. 
    # Jupyter's SPARQL kernel does not return the curie form by default.
    BIND (REPLACE(STR(?acupoint_iri), "http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#", 
                                      "TARA:") AS ?acupoint_curie)
}
ORDER BY ?meridian ?acupoint

acupoint_curie,acupoint_label,meridian,special_point_role,special_point_category
TARA:TARA_3472408,CV 17,Conception Vessel,Front-Mu Point of the Pericardium,Front-Mu Point
TARA:TARA_1205186,CV 5,Conception Vessel,Front-Mu Point of Triple Energizer,Front-Mu Point
TARA:TARA_9921528,CV 4,Conception Vessel,Front-Mu Point of the Small Intestine,Front-Mu Point
TARA:TARA_4823591,CV 14,Conception Vessel,Front-Mu Point of the Heart,Front-Mu Point
TARA:TARA_0434959,CV 12,Conception Vessel,Front-Mu Point of the Stomach,Front-Mu Point
TARA:TARA_0022762,CV 3,Conception Vessel,Front-Mu Point of the Bladder,Front-Mu Point
TARA:TARA_0214009,GB 24,Gallbladder Meridian,Front-Mu Point of the Gallbladder,Front-Mu Point
TARA:TARA_1882219,GB 25,Gallbladder Meridian,Front-Mu Point of the Kidney,Front-Mu Point
TARA:TARA_7479763,LR 14,Liver Meridian,Front-Mu Point of the Liver,Front-Mu Point
TARA:TARA_0794971,LR 13,Liver Meridian,Front-Mu Point of the Spleen,Front-Mu Point


### Q5: List all acupoints along with their meridians, surface locations, and body regions.

In [7]:
# List all the acupoints along with their meridians, surface locations, and body regions

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX TARA: <http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#>

SELECT DISTINCT 
?acupoint_curie ?acupoint_label ?meridian
?surface_region
?related_region
?body_region

WHERE 
{
   # FILTER (REGEX (?acupoint_label, 'LU')).
   # FILTER (?meridian = "Lung Meridian")
   # FILTER (?body_region = 'chest').
    FILTER (?body_region = 'limb').
   # FILTER (?surface_region = 'pedal digit 1').

    OPTIONAL {?acupoint_iri TARA:hasMeridian/rdfs:label ?meridian.}
    ?acupoint_iri rdfs:subClassOf*/rdfs:label "Meridian Acupoint".
    ?acupoint_iri rdfs:label ?acupoint_label.

    OPTIONAL 
    {   
        ?acupoint_iri TARA:hasSurfaceLocation ?surface_region_iri.
        ?surface_region_iri rdfs:label ?surface_region.
    }

    OPTIONAL
    {
        ?acupoint_iri TARA:hasRelatedLocation ?related_region_iri.
        ?related_region_iri rdfs:label ?related_region.
    }

    OPTIONAL
    {
    ?surface_region_iri ilxtr:isPartOf*/rdfs:label ?body_region.
    FILTER (?body_region in ('head', 'neck', 'back', 'chest', 'abdomen',
                             'limb', 'perineum', 'lower limb segment', 'upper limb segment', 
                             'hand', 'arm', 'face', 'leg', 'foot', 'knee')).
    }

    FILTER (?surface_region != ?related_region)

    # Exclude generic categorization classes like 'Acupoint of the X Meridian'. Meridian Acupoint, 'Special Points', etc.
    FILTER (!regex(str(?acupoint_label), 'Acupoint of|Meridian Acupoint|Point of|Point|Extra Acupoint'))
   
    # To have the curie form of the acupoint_iri in the query result. 
    # Jupyter's SPARQL kernel does not return the curie form by default.
    BIND (REPLACE(STR(?acupoint_iri), "http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#", 
                                      "TARA:") AS ?acupoint_curie) 
}
ORDER BY ?acupoint_label
limit 999

acupoint_curie,acupoint_label,meridian,surface_region,related_region,body_region
TARA:TARA_2678584,BL 37,Bladder Meridian,hindlimb stylopod,semitendinosus,limb
TARA:TARA_2678584,BL 37,Bladder Meridian,hindlimb stylopod,gluteal sulcus,limb
TARA:TARA_2678584,BL 37,Bladder Meridian,hindlimb stylopod,biceps femoris,limb
TARA:TARA_3339722,BL 38,Bladder Meridian,knee,biceps femoris,limb
TARA:TARA_3339722,BL 38,Bladder Meridian,knee,biceps femoris tendon,limb
TARA:TARA_3339722,BL 38,Bladder Meridian,knee,popliteal crease,limb
TARA:TARA_2186848,BL 39,Bladder Meridian,knee,biceps femoris tendon,limb
TARA:TARA_2186848,BL 39,Bladder Meridian,knee,popliteal crease,limb
TARA:TARA_5290080,BL 40,Bladder Meridian,knee,popliteal crease,limb
TARA:TARA_4648540,BL 55,Bladder Meridian,hindlimb zeugopod,gastrocnemius,limb


### Q6: List all acupoints and optionally show the conditions treated using those acupoint.
This query result includes everything needed to answer following types of questions:
* What acupoints are used to treat certain conditions (e.g., pain-related conditions)? 
    * What meridians do they belong? What special point roles do they have, if any?
    * Where on the body surface the acupoints are located in? Show the acupoints based on their location on different regions of the body. (e.g., what acupoints are located on the head that can be used to treat headech?
* What specific pain conditions are studied using those ocupoints?
    * provide the article references where those acupoints were used.
        * include the needling procedure, stimulation type, and acupuncture modality, etc.
        * In which country the study was conducted? What was the sample size and controls?
* What special points can be used to treat pain conditions? What kind of pains can they be used to treat?


In [2]:
# List all the acupoints along with their meridians, special point roles, surface locations, and body regions. 
# Also include the conditions that can be treated using those acupoints including provinance details.
# Additionally, include the study details.

PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
PREFIX TARA: <http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#>

SELECT DISTINCT 
?Acupint_IRI ?Acupoint
?Acupint_Category_IRI ?Acupoint_Category
?Meridian_IRI ?Meridian

?Special_Point_Role_IRI ?Special_Point_Role
?Special_Point_Category_IRI ?Special_Point_Category

?Surface_Region_IRI ?Surface_Region
?Related_Region_IRI ?Related_Region
?Body_Region_IRI ?Body_Region

?Studied_Article_IRI ?Studied_Article_Title
?Authors ?Publication_Venue ?Publication_Date
?Country ?Trial_Type ?Modality ?Stimulation_Type
?Needling_Info ?Sample_Size ?Controls ?Trial_Type
?Listed_Acupoints

?Studied_Condition_IRI ?Studied_Condition ?Condition_Note
?Condition_Context_IRI ?Condition_Context

WHERE 
{
   # FILTER (REGEX (?Condition_Context, 'Pain')).
   # FILTER (REGEX (?Acupoint, 'LU')).
   # FILTER (REGEX (?Acupoint_Category, 'Extra')).
   # FILTER (?Meridian = "Main Meridian")
   # FILTER (?Body_Region = 'face').
   # FILTER (?Surface_Region = 'pedal digit 1').
    FILTER (?Studied_Article_IRI != ''). # only show the results with articles data

    
    ?Acupint_IRI rdfs:subClassOf+/rdfs:label "Acupoint".
    ?Acupint_IRI rdfs:label ?Acupoint.    
    
    OPTIONAL
    {
      ?Acupint_IRI rdfs:subClassOf* ?Acupint_Category_IRI.
      ?Acupint_Category_IRI rdfs:label ?Acupoint_Category.
      FILTER (?Acupoint_Category in ('Meridina Acupoint', 'Extra Acupoint' , 'Special Point')).
    }
    
    OPTIONAL 
    {
        ?Acupint_IRI TARA:hasMeridian ?Meridian_IRI.
        ?Meridian_IRI rdfs:subClassOf*/rdfs:label "Meridian".
        ?Meridian_IRI rdfs:label ?Meridian.
    }

    OPTIONAL 
    {
        ?Acupint_IRI TARA:hasDesignatedSpecialPointRole ?Special_Point_Role_IRI. 
        ?Special_Point_Role_IRI rdfs:label ?Special_Point_Role. 

        ?Special_Point_Role_IRI rdfs:subClassOf* ?Special_Point_Category_IRI.
        ?Special_Point_Category_IRI rdfs:subClassOf*/rdfs:label 'Special Point'.
        
        ?Special_Point_Category_IRI rdfs:label ?Special_Point_Category.
    }
    
  OPTIONAL
  {
        ?Acupint_IRI TARA:hasSurfaceLocation ?Surface_Region_IRI.
        ?Surface_Region_IRI rdfs:label ?Surface_Region.
        
        OPTIONAL
        {
            ?Acupint_IRI TARA:hasRelatedLocation ?Related_Region_IRI.
            ?Related_Region_IRI rdfs:label ?Related_Region.
        }

        OPTIONAL
        {
            ?Surface_Region_IRI ilxtr:isPartOf* ?Body_Region_IRI.
            ?Body_Region_IRI rdfs:label ?Body_Region.
            FILTER (?Body_Region in ('head', 'neck', 'back', 'chest', 'abdomen',
                                    'limb', 'perineum', 'lower limb segment', 'upper limb segment', 
                                    'hand', 'arm', 'face', 'leg', 'foot', 'knee')).

        }
    FILTER (?Surface_Region != ?Related_Region)
  }

   OPTIONAL
    {
        ?Acupint_IRI TARA:isStudiedInArticle ?Studied_Article_IRI.
        ?Studied_Article_IRI dc:title ?Studied_Article_Title;
                             TARA:hasAuthor ?Authors.
        
        OPTIONAL {?Studied_Article_IRI TARA:hasPublicationVenue ?Publication_Venue.}
        OPTIONAL {?Studied_Article_IRI TARA:hasPublicationDate ?Publication_Date.}
        OPTIONAL {?Studied_Article_IRI  TARA:hasCountryInformation ?Country.}

        OPTIONAL {?Studied_Article_IRI  TARA:hasTrialType ?Trial_Type.}

        OPTIONAL {?Studied_Article_IRI  TARA:hasAcupunctureModality ?Modality.}
        
        OPTIONAL {?Studied_Article_IRI  TARA:hasStimulationType ?Stimulation_Type.}
        OPTIONAL {?Studied_Article_IRI  TARA:hasNeedlingInformation ?Needling_Info.}
        OPTIONAL {?Studied_Article_IRI  TARA:hasSampleSizeInformation ?Sample_Size.}
        OPTIONAL {?Studied_Article_IRI  TARA:hasControlsInformation ?Controls.}
        
        OPTIONAL {?Studied_Article_IRI  TARA:hasListedAcupointsUsed ?Listed_Acupoints.}
        
        OPTIONAL {?Studied_Article_IRI TARA:hasStudiedCondition ?Studied_Condition_IRI.
                  ?Studied_Condition_IRI  rdfs:label ?Studied_Condition.}
        
        OPTIONAL {?Studied_Article_IRI TARA:hasStudiedConditionNote ?Condition_Note.}
        
        OPTIONAL {?Studied_Article_IRI TARA:hasStudiedConditionContext ?Condition_Context_IRI.
                  ?Condition_Context_IRI rdfs:label ?Condition_Context.}

    }

    # Exclude generic categorization classes like 'Acupoint of the X Meridian'. Meridian Acupoint, 'Special Points', etc.
    FILTER (!regex(str(?Acupoint), 'Acupoint of|Meridian|Point of|Point|Extra Acupoint'))
   
    # # To have the curie form of the acupoint_iri in the query result. 
    # # Jupyter's SPARQL kernel does not return the curie form by default.
    # BIND (REPLACE(STR(?Acupint_IRI), "http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#", 
    #                                   "TARA:") AS ?Acupoint_curie) 
}
ORDER BY 
    DESC(?Publication_Date) ?Acupoint ?Meridian ?Special_Point_Role ?Special_Point_Category 
    ?Surface_Region ?Related_Region ?Body_Region
limit 9999

Acupint_IRI,Acupoint,Acupint_Category_IRI,Acupoint_Category,Meridian_IRI,Meridian,Special_Point_Role_IRI,Special_Point_Role,Special_Point_Category_IRI,Special_Point_Category,Surface_Region_IRI,Surface_Region,Related_Region_IRI,Related_Region,Body_Region_IRI,Body_Region,Studied_Article_IRI,Studied_Article_Title,Authors,Publication_Venue,Publication_Date,Country,Trial_Type,Modality,Stimulation_Type,Needling_Info,Sample_Size,Controls,Listed_Acupoints,Studied_Condition_IRI,Studied_Condition,Condition_Note,Condition_Context_IRI,Condition_Context
http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#TARA_0992296,BL 23,http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#TARA_7852472,Special Point,http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#TARA_7525476,Bladder Meridian,http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#TARA_2432731,Back-Shu Point of the Kidney,http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#TARA_3861175,Back-Shu Point,http://purl.obolibrary.org/obo/UBERON_0005462,lower back,http://purl.obolibrary.org/obo/UBERON_0004618,lumbar vertebra 2,,,https://doi.org/10.1371/journal.pone.0000919,Naturopathic Care for Chronic Low Back Pain: A Randomized Trial,"Orest Szczurko, Kieran Cooley, Jason W Busse, Dugald Seely, Bob Bernhardt, Gordon H Guyatt, Qi Zhou, Edward J Mills",PLoS ONE,2007,Canada,RCT,"The acupuncture modality involved twice-weekly sessions for 12 weeks, using Seirin disposable needles number 5 (0.25630 mm) inserted at specific points (GV 3,4, BL 23, 25, 40 bilaterally) to a depth of 0.5 cun, with stimulation to achieve de qi and left in place for 20 minutes per session.",Manual stimulation to achieve de qi sensation; needles left in place for 20 minutes,Needle depth: 0.5 cun; Needle angle: Not mentioned,Total: 75 participants; Naturopathic care group: 39 participants; Control group: 36 participants,"The control group received standardized physiotherapy advice, which included an educational booklet designed by the British Physiotherapy Association. This booklet provided information on back pain management, including causes, prognosis, appropriate use of imaging studies and specialists, and exercises for promoting recovery and preventing recurrences. Participants were also instructed on specific back stretching and strengthening exercises and relaxation techniques.","GV 3, GV 4, BL 23, BL 25, BL 40",http://purl.obolibrary.org/obo/HP_0003419,Low back pain,chronic low back pain,http://purl.obolibrary.org/obo/HP_0012531,Pain
http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#TARA_0992296,BL 23,http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#TARA_7852472,Special Point,http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#TARA_7525476,Bladder Meridian,http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#TARA_2432731,Back-Shu Point of the Kidney,http://www.acupunctureresearch.org/tara/ontology/acupoints.owl#TARA_3861175,Back-Shu Point,http://purl.obolibrary.org/obo/UBERON_0005462,lower back,http://uri.interlex.org/base/ilx_0795312,posterior median line,,,https://doi.org/10.1371/journal.pone.0000919,Naturopathic Care for Chronic Low Back Pain: A Randomized Trial,"Orest Szczurko, Kieran Cooley, Jason W Busse, Dugald Seely, Bob Bernhardt, Gordon H Guyatt, Qi Zhou, Edward J Mills",PLoS ONE,2007,Canada,RCT,"The acupuncture modality involved twice-weekly sessions for 12 weeks, using Seirin disposable needles number 5 (0.25630 mm) inserted at specific points (GV 3,4, BL 23, 25, 40 bilaterally) to a depth of 0.5 cun, with stimulation to achieve de qi and left in place for 20 minutes per session.",Manual stimulation to achieve de qi sensation; needles left in place for 20 minutes,Needle depth: 0.5 cun; Needle angle: Not mentioned,Total: 75 participants; Naturopathic care group: 39 participants; Control group: 36 participants,"The control group received standardized physiotherapy advice, which included an educational bookl